# **Maestría en Inteligencia Artificial Aplicada**

## Curso: **Análisis de grandes volúmenes de datos (Gpo 10)**

### Tecnológico de Monterrey

## **Actividad 4**

###  *Métricas de calidad de resultados*


#### **Nombrey matrícula**

*   **A01746998** - Alexys Martín Coate Reyes


# **1. Construcción de la muestra M**



### Cargando los datos

In [ ]:
# Librerias
from pyspark.sql import SparkSession

In [ ]:
# Crear sesión Spark
spark = SparkSession.builder \
    .appName("EDA_Vuelos") \
    .getOrCreate()

# Leer el CSV
df = spark.read.csv("../Dataset/Airline_Delay_2016-2018.csv", header=True, inferSchema=True)

# Mostrar esquema de datos
df.printSchema()

# Número total de registros
total_registros = df.count()
print(f"Número total de registros: {total_registros}")

root
 |-- FL_DATE: date (nullable = true)
 |-- OP_CARRIER: string (nullable = true)
 |-- OP_CARRIER_FL_NUM: integer (nullable = true)
 |-- ORIGIN: string (nullable = true)
 |-- DEST: string (nullable = true)
 |-- CRS_DEP_TIME: integer (nullable = true)
 |-- DEP_TIME: double (nullable = true)
 |-- DEP_DELAY: double (nullable = true)
 |-- TAXI_OUT: double (nullable = true)
 |-- WHEELS_OFF: double (nullable = true)
 |-- WHEELS_ON: double (nullable = true)
 |-- TAXI_IN: double (nullable = true)
 |-- CRS_ARR_TIME: integer (nullable = true)
 |-- ARR_TIME: double (nullable = true)
 |-- ARR_DELAY: double (nullable = true)
 |-- CANCELLED: double (nullable = true)
 |-- DIVERTED: double (nullable = true)
 |-- CRS_ELAPSED_TIME: double (nullable = true)
 |-- ACTUAL_ELAPSED_TIME: double (nullable = true)
 |-- AIR_TIME: double (nullable = true)
 |-- DISTANCE: double (nullable = true)

Número total de registros: 18505725


In [ ]:
# Imprimiendo los 3 primeros rengloes del dataframe dataframe
df.show(3)

+----------+----------+-----------------+------+----+------------+--------+---------+--------+----------+---------+-------+------------+--------+---------+---------+--------+----------------+-------------------+--------+--------+
|   FL_DATE|OP_CARRIER|OP_CARRIER_FL_NUM|ORIGIN|DEST|CRS_DEP_TIME|DEP_TIME|DEP_DELAY|TAXI_OUT|WHEELS_OFF|WHEELS_ON|TAXI_IN|CRS_ARR_TIME|ARR_TIME|ARR_DELAY|CANCELLED|DIVERTED|CRS_ELAPSED_TIME|ACTUAL_ELAPSED_TIME|AIR_TIME|DISTANCE|
+----------+----------+-----------------+------+----+------------+--------+---------+--------+----------+---------+-------+------------+--------+---------+---------+--------+----------------+-------------------+--------+--------+
|2016-01-01|        DL|             1248|   DTW| LAX|        1935|  1935.0|      0.0|    23.0|    1958.0|   2107.0|   13.0|        2144|  2120.0|    -24.0|      0.0|     0.0|           309.0|              285.0|   249.0|  1979.0|
|2016-01-01|        DL|             1251|   ATL| GRR|        2125|  2130.0|     

### Seleccionando variables de caracterización

In [ ]:
# Variables seleccionadas
vars_particion = ["OP_CARRIER", "ORIGIN", "DEST", "CANCELLED", "DIVERTED"]

# Imrpimiendo la cantidad de valores únicos que se tiene por las columnas seleccionadas
for col in vars_particion:
    print(col, df.select(col).distinct().count())

OP_CARRIER 18
ORIGIN 362
DEST 360
CANCELLED 2
DIVERTED 2


In [ ]:
from pyspark.sql.functions import col, count, round

# Calcular frecuencia de combinaciones
combinaciones = df.groupBy(vars_particion).count()

# Total de registros
total = df.count()

# Agregar probabilidad
combinaciones = combinaciones.withColumn("probabilidad", col("count") / total)

combinaciones.show(truncate=False)

+----------+------+----+---------+--------+-----+---------------------+
|OP_CARRIER|ORIGIN|DEST|CANCELLED|DIVERTED|count|probabilidad         |
+----------+------+----+---------+--------+-----+---------------------+
|EV        |ATL   |AGS |0.0      |0.0     |2912 |1.573567098830227E-4 |
|EV        |DTW   |BWI |0.0      |0.0     |23   |1.2428586288837644E-6|
|EV        |BRO   |IAH |0.0      |0.0     |3054 |1.6503001098308766E-4|
|F9        |DEN   |MIA |0.0      |0.0     |1089 |5.884665421106171E-5 |
|WN        |BDL   |RSW |0.0      |0.0     |735  |3.971743879258986E-5 |
|WN        |BWI   |MEM |0.0      |0.0     |1348 |7.284232311892671E-5 |
|WN        |DEN   |BOS |0.0      |0.0     |2467 |1.3331009728070637E-4|
|WN        |LAX   |LAS |0.0      |0.0     |10922|5.901957367247162E-4 |
|WN        |LAX   |SAT |0.0      |0.0     |2056 |1.1110075395587041E-4|
|WN        |MCO   |CMH |0.0      |0.0     |3558 |1.922648261551493E-4 |
|WN        |PIT   |DEN |0.0      |0.0     |1307 |7.0626792519612

In [ ]:
from pyspark.sql.functions import when, lit

# Definir tamaño total de la muestra (por ejemplo, 1% del total)
tamaño_muestra_total = int(total * 0.01)

# Establecer un mínimo de registros por partición
minimo_por_particion = 0

# Calcular el tamaño de muestra por partición según su probabilidad
combinaciones_con_tamaño = combinaciones.withColumn(
    "tamaño_muestra",
    round(
        when(
            (col("probabilidad") * tamaño_muestra_total) < minimo_por_particion,
            lit(minimo_por_particion)
        ).otherwise(col("probabilidad") * tamaño_muestra_total)
    ).cast("int")
)

# Ordenar para visualizar mejor
combinaciones_con_tamaño = combinaciones_con_tamaño.orderBy(col("tamaño_muestra").desc())

combinaciones_con_tamaño.show(10, truncate=False)

# Calcular el tamaño final total de la muestra
tamaño_muestra_final = combinaciones_con_tamaño.agg({"tamaño_muestra": "sum"}).collect()[0][0]

print(f"Tamaño total esperado de la muestra final: {tamaño_muestra_final}")

+----------+------+----+---------+--------+-----+---------------------+--------------+
|OP_CARRIER|ORIGIN|DEST|CANCELLED|DIVERTED|count|probabilidad         |tamaño_muestra|
+----------+------+----+---------+--------+-----+---------------------+--------------+
|HA        |OGG   |HNL |0.0      |0.0     |28891|0.0015611925498730799|289           |
|HA        |HNL   |OGG |0.0      |0.0     |28830|0.0015578962726399534|288           |
|HA        |KOA   |HNL |0.0      |0.0     |21350|0.0011536970315942769|213           |
|HA        |HNL   |KOA |0.0      |0.0     |20673|0.0011171137580397417|207           |
|HA        |HNL   |LIH |0.0      |0.0     |19946|0.0010778286179006767|199           |
|HA        |LIH   |HNL |0.0      |0.0     |19901|0.0010753969379745998|199           |
|WN        |DAL   |HOU |0.0      |0.0     |19109|0.0010325993712756458|191           |
|WN        |HOU   |DAL |0.0      |0.0     |19131|0.0010337881925728389|191           |
|DL        |MCO   |ATL |0.0      |0.0     |

In [ ]:
from pyspark.sql.window import Window
from pyspark.sql.functions import row_number

# Unir el tamaño de muestra a cada combinación en el dataset original
df_con_muestra = df.join(
    combinaciones_con_tamaño.select(*vars_particion, "tamaño_muestra"),
    on=vars_particion,
    how="inner"
)

# Crear ventana ordenada por fecha y hora dentro de cada combinación
ventana = Window.partitionBy(*vars_particion).orderBy("FL_DATE", "CRS_DEP_TIME")

# Enumerar vuelos por combinación (ordenados por tiempo)
df_con_muestra = df_con_muestra.withColumn("row_num", row_number().over(ventana))

# Filtrar solo los primeros N registros por combinación
muestra_final = df_con_muestra.filter(col("row_num") <= col("tamaño_muestra"))

# Mostrar la muestra final
muestra_final.show(10, truncate=False)

print(f"Tamaño total de la muestra: {muestra_final.count()}")

+----------+------+----+---------+--------+----------+-----------------+------------+--------+---------+--------+----------+---------+-------+------------+--------+---------+----------------+-------------------+--------+--------+--------------+-------+
|OP_CARRIER|ORIGIN|DEST|CANCELLED|DIVERTED|FL_DATE   |OP_CARRIER_FL_NUM|CRS_DEP_TIME|DEP_TIME|DEP_DELAY|TAXI_OUT|WHEELS_OFF|WHEELS_ON|TAXI_IN|CRS_ARR_TIME|ARR_TIME|ARR_DELAY|CRS_ELAPSED_TIME|ACTUAL_ELAPSED_TIME|AIR_TIME|DISTANCE|tamaño_muestra|row_num|
+----------+------+----+---------+--------+----------+-----------------+------------+--------+---------+--------+----------+---------+-------+------------+--------+---------+----------------+-------------------+--------+--------+--------------+-------+
|9E        |ATL   |AGS |0.0      |0.0     |2018-01-01|3296             |855         |854.0   |-1.0     |12.0    |906.0     |939.0    |3.0    |953         |942.0   |-11.0    |58.0            |48.0               |33.0    |143.0   |10          

# **2. Construcción Train – Test**



# **3.  Selección de métricas para medir calidad de resultados**



# **4. Entrenamiento de Modelos de Aprendizaje**



# **5. Análisis de resultados**

